In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))

import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel, AutoModel
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch
from torch.optim import AdamW
# 
from src.data.dataset import EssayDataset
from src.models.bert_regression import BertRegressionModel

### Preprocessing

In [3]:
df = pd.read_csv("../data/aes_dataset.csv")
df.head()

,question,reference_answer,answer,score,dataset,max_length1,normalized_score,normalized_score2
0,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,"sumber tenaga, pemanis alami, menjaga sistem i...",27.0,analisis_essay,65,0.27,27
1,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,"sebagai sumber energi, pemanis alami, menjaga ...",21.0,analisis_essay,66,0.21,21
2,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,1. Sebagai energi. 2. Sebagai memperlancaar pe...,42.0,analisis_essay,76,0.42,42
3,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,"untuk membuat kenyang, agar tidak lapar, agar ...",18.0,analisis_essay,67,0.18,18
4,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,Karbohidrat mempunyai peran penting untuk pros...,82.0,analisis_essay,105,0.82,82


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22406 entries, 0 to 22405
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   question          4859 non-null   object 
 1   reference_answer  22406 non-null  object 
 2   answer            22406 non-null  object 
 3   score             22406 non-null  float64
 4   dataset           22406 non-null  object 
dtypes: float64(1), object(4)
memory usage: 875.4+ KB


In [4]:
df['dataset'].value_counts()

dataset
asap              17043
sag                2558
analisis_essay     2162
stita               333
cunlp               171
sci                 139
Name: count, dtype: int64

In [5]:
# convert NaN to None
df['question'] = df['question'].replace({np.nan: None})

In [6]:
tokenizer1 = BertTokenizer.from_pretrained("indobenchmark/indobert-lite-base-p2")
dataset = EssayDataset(df, tokenizer1, 512, "ALBERT")

df['max_length1'] = df.index.map(dataset.get_max_length)
len(df[(df['dataset'] != 'analisis_essay')&(df['max_length1'] > 510)])

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


308

In [7]:
df['max_length1'].describe()

count    22406.000000
mean       161.063688
std        108.950303
min         30.000000
25%        114.000000
50%        150.000000
75%        186.000000
max       1832.000000
Name: max_length1, dtype: float64

## Normalize Score

In [8]:
def normalize_score(row, min_max_dict):
    min_score, max_score = min_max_dict[row['dataset']]
    return (row['score'] - min_score) / (max_score - min_score)

# Menghitung min dan max per kategori sebagai tuple
min_max_dict = df.groupby('dataset')['score'].agg(['min', 'max']).apply(tuple, axis=1).to_dict()

# Menambahkan kolom normalized_score
df['normalized_score'] = df.apply(lambda x: normalize_score(x, min_max_dict), axis=1)


In [9]:
df['normalized_score'].describe()

count    22406.000000
mean         0.384133
std          0.339251
min          0.000000
25%          0.000000
50%          0.333333
75%          0.666667
max          1.000000
Name: normalized_score, dtype: float64

## Training

In [10]:
df_sag = df[df['dataset'] == 'sag']
df_sag.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2558 entries, 19376 to 21933
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   question          2558 non-null   object 
 1   reference_answer  2558 non-null   object 
 2   answer            2558 non-null   object 
 3   score             2558 non-null   float64
 4   dataset           2558 non-null   object 
 5   max_length1       2558 non-null   int64  
 6   normalized_score  2558 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 159.9+ KB


In [11]:
df_test = df_sag[:12]
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 19376 to 19387
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   question          12 non-null     object 
 1   reference_answer  12 non-null     object 
 2   answer            12 non-null     object 
 3   score             12 non-null     float64
 4   dataset           12 non-null     object 
 5   max_length1       12 non-null     int64  
 6   normalized_score  12 non-null     float64
dtypes: float64(2), int64(1), object(4)
memory usage: 768.0+ bytes


In [12]:
test_dataset = EssayDataset(df_test, tokenizer1, 512, "ALBERT")

In [13]:
dataloader = DataLoader(test_dataset, batch_size=2)

In [14]:
model = BertRegressionModel("indobenchmark/indobert-lite-base-p2")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [15]:
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.MSELoss()
model.train()

BertRegressionModel(
  (bert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertSdpaAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True

In [16]:
for epoch in range(1):
    for batch in dataloader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        scores = batch['scores'].to(device)

        # Forward pass
        predictions = model(input_ids=input_ids, attention_mask=attention_mask).squeeze()

        # Compute loss
        loss = criterion(predictions, scores)
        print("loss", loss.item())
        # Backward pass
        loss.backward()
        optimizer.step()


c:\Users\User\Documents\Code\env\lib\site-packages\transformers\models\albert\modeling_albert.py:404: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attention_output = torch.nn.functional.scaled_dot_product_attention(


loss 0.5393558144569397
loss 0.04712935537099838
loss 0.6701321005821228
loss 0.2333250194787979
loss 0.03378669172525406
loss 0.04550880938768387
